# Uses interaction to push triples to the brain and query it

This notebook assume you have knowledge about GAF, GRasP and roboGRasP. These are explained in the following papers:

Fokkens, Antske, Marieke Van Erp, Piek Vossen, Sara Tonelli, Willem Robert Van Hage, Luciano Serafini, Rachele Sprugnoli, and Jesper Hoeksema. "GAF: A grounded annotation framework for events." In Workshop on Events: Definition, Detection, Coreference, and Representation, pp. 11-20. 2013: Fokkens-etal-2013-GAF.pdf  Download Fokkens-etal-2013-GAF.pdf

Fokkens, Antske, Piek Vossen, Marco Rospocher, Rinke Hoekstra, Willem R. van Hage, and Fondazione Bruno Kessler. "Grasp: grounded representation and source perspective." In Proceedings of the Workshop Knowledge Resources for the Socio-Economic Sciences and Humanities associated with RANLP 2017, pp. 19-25. 2017: Fokkens-etal-2017-grasp-KnowRSH-2017.pdf  Download Fokkens-etal-2017-grasp-KnowRSH-2017.pdf


Santamaría, Selene Báez, Thomas Baier, Taewoon Kim, Lea Krause, Jaap Kruijt, and Piek Vossen. "EMISSOR: A platform for capturing multimodal interactions as Episodic Memories and Interpretations with Situated Scenario-based Ontological References." arXiv preprint arXiv:2105.08388 (2021): https://arxiv.org/pdf/2105.08388.pdf

The GRaSP model is defined here: https://github.com/cltl/GRaSP

Before running, start GraphDB and make sure that there is a sandbox repository.
GraphDB can be downloaded from:

https://graphdb.ontotext.com


In [1]:
import json
import os
import time
import uuid
from datetime import date
from datetime import datetime
from random import getrandbits, choice
import pathlib

# general imports for EMISSOR and the BRAIN
import emissor as em
import requests
from cltl import brain
from cltl.brain.long_term_memory import LongTermMemory
from cltl.brain.utils.helper_functions import brain_response_to_json
from cltl.combot.backend.api.discrete import UtteranceType
from cltl.reply_generation.data.sentences import GREETING, ASK_NAME, ELOQUENCE, TALK_TO_ME
from cltl.reply_generation.lenka_replier import LenkaReplier
from cltl.triple_extraction.api import Chat, UtteranceHypothesis

/Users/piek/PycharmProjects/cltl-chatbots/venv/lib/python3.7/site-packages/rdflib_jsonld/__init__.py:12: DeprecationWarning: The rdflib-jsonld package has been integrated into rdflib as of rdflib==6.0.1.  Please remove rdflib-jsonld from your project's dependencies.
  DeprecationWarning,
[nltk_data] Downloading package punkt to /Users/piek/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Importing the chatbot utility functions

In [2]:
import sys
import os

src_path = os.path.abspath(os.path.join('..'))
if src_path not in sys.path:
    sys.path.append(src_path)

#### The next utils are needed for the interaction and creating triples and capsules
import chatbots.util.driver_util as d_util
import chatbots.util.capsule_util as c_util
import chatbots.intentions.talk as talk

## Specifying the BRAIN

We imported the brain from CLTL. We create an instance using the class *LongTermMemory*, which takes three parameters: 1) the address of the triple store, 2) a path to a folder for logging the triples and a boolean flag for clearing the repository and reload the initial ontologies.

As the address of the triple store, we give the GraphDB localhost port (7200) and specify the name of the repository that we created beforehand. Any repository will do. We use here the *sandbox* repository. Note that you can also specify a remote SPARQL endpoint of another triple store or share a triple store among systems.

For storing the triples generate, we define a scenario folder in a data folder relative to where the notebooks are stored. It is based on the timestamp when we start. We will use this scenario structure later for the interaction as well as we did before.

Finally, if you set *clear_all* to *True*, the sandbox triple store is emptied (memory erased) and the basic ontological models are reloaded. Setting it to *False* means you add things to the current memory.

In [3]:
### The name of your scenario
scenario_id = datetime.today().strftime("%Y-%m-%d-%H:%M:%S")

### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
scenario_path = os.path.abspath(os.path.join('../../data'))
if scenario_path not in sys.path:
    sys.path.append(scenario_path)

    ### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
scenario_path = os.path.abspath(os.path.join('../../data'))
if scenario_path not in sys.path:
    sys.path.append(scenario_path)

if not os.path.exists(scenario_path) :
    os.mkdir(scenario_path)
    print("Created a data folder for storing the scenarios", scenario_path)
    
rdffolder = scenario_path + "/" + scenario_id + "/" + "rdf"
log_path = pathlib.Path(rdffolder)
my_brain = brain.LongTermMemory(address="http://localhost:7200/repositories/sandbox",
                                log_dir=log_path,
                                clear_all=True)


2021-11-15 14:45:49,557 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Booted
2021-11-15 14:45:53,991 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Uploading ontology to brain
2021-11-15 14:45:56,015 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Booted
2021-11-15 14:45:56,017 -     INFO -  cltl.brain.basic_brain.LocationReasoner - Booted
2021-11-15 14:45:56,019 -     INFO -      cltl.brain.basic_brain.TypeReasoner - Booted
2021-11-15 14:45:56,021 -     INFO -   cltl.brain.basic_brain.TrustCalculator - Booted
2021-11-15 14:45:56,128 -     INFO -   cltl.brain.basic_brain.TrustCalculator - Computed trust for all known agents


### Posting triples to the brain

The *talk.py* script in *intentions* demonstrates the basic API functions for posting and querying the BRAIN. The input of for these function is a triple in JSON format. In the next cell, we define such a triple for the subject 'Fred, the predicate 'like' and the object 'singing'. In addition to a *label* that is used to identify the resource URI in the knowledge graph, we also need to provide type information.


In [4]:
test_triple = {'subject': {'label': 'Karla', 'type': ['noun.person']},
               'predicate': {'label': 'like', 'type': ['verb.emotion']},
               'object': {'label': 'singing', 'type': ['noun.act']}}

The BRAIN uses the roboGRaSP model to capture knowledge. In roboGRaSP, we keep track of the *mentioning* of knowledge in signals. This can be either through conversation or though perception. Whenever a signal makes reference to knowledge (e.g. people, objects, properties), there is also a source of the signal and possibly a perspective.
Sources are typically speakers with whom a system interacts but they can also be the camera or microphone of the agent that picked up a signal. Perspectives reflect the attitude and appraisal of the source towards the triple. Possible perspective values are, among others: certainty, sentiment, emotion, epistemic belief, deontic judgement, ethical judgment. Following GRaSP, the actual triple is reprsented as a claim made by the source in or through a signal and the perspective values are represented as attributions of the source.

In order to deal with these GRaSP layers, we need to embed the *factual* triple within a so-called *capsule* JSON structures that provides contextual information, the source and the perspective. The following attributes are required:

* The physical context is time and space: context_id, contry, region, city, place, place_id, position, date
* The physical objects in the physical context: objects, people
* The interactive context: chat, turn, utterance and utterance type 
* The author (source)
* The triple: subject, predicate and object
* The perspective

Next is a capsule example, that contains the above triple embedded within the minimal contextual information:

```
{'context_id': '1',
  'country': '',
  'city': '',
  'region': '',
  'place': '',
  'place_id': '',
  'position': '',
  'date': datetime.date(2021, 11, 9),
  'objects': [],
  'people': [],
  'author': 'me',
  'chat': '1',
  'turn': '1',
  'utterance': '',
  'utterance_type': <UtteranceType.STATEMENT: 0>
  'subject': {'label': 'Fred', 'type': ['agent']},
  'predicate': {'label': 'like', 'type': ['verb.emotion']},
  'object': {'label': 'Fred', 'type': ['noun.object']},
  'perspective': []
}
```

The *capsule_util.py* within util has a number of functions to create capsules. We use these functions within the different intentions that involve the brain, as shown below.

To post the above triple as a simple statement we use the *post_a_triple_and_get_thoughts* function from the *talk.py* that you find within the *intentions* module. This function takes a triple and an initialised brain as parameters and it returns the capsule that is created but also the response from the brain as a JSON structure. 

Posting triples to the brain is done through the *update* API function. This takes three parameters: the capsule and two boolean settings: *reason_types*  triggers linking subjects and entities to their types by consulting the semantic web, and *create_label* which triggers the system to create an additional *rdfs:label* property from the subject and object label if it is an entity. We typically set these to false, which are also the default values. Calling the update function not only stores the triple but is also followed by a series of preprogrammed SPAQRL queries that represent the *thoughts* on the changes to the brain.

In [5]:
import pprint


capsule, throughts_json = talk.post_a_triple_label_and_type(test_triple, my_brain)
print("Capsule that provides the interactive contexts for the signal to which a triple is grouned:\n")
pprint.pprint(capsule)


2021-11-15 14:46:03,033 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in statement: karla_like_singing [person_->_act])
2021-11-15 14:46:03,073 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Entity Novelty: new subject - new object 
2021-11-15 14:46:04,725 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Negation Conflicts: me on November,2021 about UNDERSPECIFIED
2021-11-15 14:46:04,760 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Gaps: 26 gaps as subject: e.g. be-ancestor-of person - 15 gaps as object: e.g. be-family-of person
Capsule that provides the interactive contexts for the signal to which a triple is grouned:

{'author': 'me',
 'chat': '1',
 'city': '',
 'context_id': '1',
 'country': '',
 'date': datetime.date(2021, 11, 15),
 'object': {'label': 'singing', 'type': ['noun.act']},
 'objects': [],
 'people': [],
 'perspective': <cltl.brain.infrastructure.api.Perspective object at 0x7fbc38368bd0>,
 'place': '',
 'place_id': '',
 'posit

In the above print outs, we see the triple echoed in the capsule. After posting the triple, we could query the brain in GraphDB as is shown in the next image:

![graph-db query](images/graph-db-query-1.png "GraphDB query for the triple result")

The *post_a_triple_label_and_type* function used the API call *my_brain.update(capsule, reason_types=True, create_label=True)*. The parameter *reason_types=True* triggers the brain to add type information to instances, possibly by consulting the semantic web, whereas the parameter *createLabel=true* adds the *rdfs:label* property on the basis of the labels. This is shown in the next screen dump from GraphDB:

![graph-db properties for fred](images/graph-db-properties-3.png "GraphDB showing addition properties derived for Fred")



Note that the mentioning of Fred through the capsule resulted in adding a *gaf:denotedIn* property that points to the chat and utterance identifiers that were specified (so far dummay values) and an additional *type* to *gaf:Instance*.

### Getting the thoughts

We also caught the response of the brain to this new information, which looks as shown below. The *thoughts* element lists different possible issues: _complement_conflict,_complement_gaps, _entity_novelty, _negation_conflicts, _overlaps, _statement_novelty, _subject_gaps, trust. Since we have not added a lot of data, most elements are still empty.

In [6]:
print()
print("Thoughts resulting from posting the triples to the brain:\n")
pprint.pprint(throughts_json)


Thoughts resulting from posting the triples to the brain:

{'response': '204',
 'statement': {'author': 'me',
               'chat': '1',
               'city': '',
               'context_id': '1',
               'country': '',
               'date': '2021-11-15',
               'object': {'label': 'singing', 'type': ['noun.act']},
               'objects': [],
               'people': [],
               'perspective': {'_certainty': 'UNDERSPECIFIED',
                               '_emotion': 'UNDERSPECIFIED',
                               '_polarity': 'UNDERSPECIFIED',
                               '_sentiment': 'UNDERSPECIFIED',
                               '_time': None},
               'place': '',
               'place_id': '',
               'position': '',
               'predicate': {'label': 'like', 'type': 'verb.emotion'},
               'region': '',
               'subject': {'label': 'Karla', 'type': ['noun.person']},
               'triple': {'_complement': {'_conf

We can see that most thoughts are empty because the brain has hardly been populated but there are a lot of subject_gaps based on the ontology that assume that persons can have these properties.

To verbalise the thoughts. we use a replier *LenkaReplier* imported from *cltl.reply_generation*. This replier randomly selects a thought and uses templates to generate a natural language phrase.

In [7]:
replier = LenkaReplier()
reply = replier.reply_to_statement(throughts_json, proactive=True, persist=True)
print(reply)

2021-11-15 14:46:23,743 -     INFO -   cltl.reply_generation.api.LenkaReplier - Booted
Interesting! I did not know anybody who like singing


You can call the replier repititively for the same response and it may select a different thought or phrasing of a thought.

In [8]:
for i in range(10):
    reply = replier.reply_to_statement(throughts_json, proactive=True, persist=True)
    print(reply)

I am curious. Has karla ever like a agent?
I am curious. Has karla experience taste?
Let me ask you something. Has karla ever be friends with a person?
This is news to me. I did not know anybody who like singing
I am curious. Has karla ever read by a book?
I would like to know. Has karla study at institution?
This is news to me. I did not know anybody who like singing
I would like to know. What types can karla like
This is news to me. I did not know anything that karla like
If you don't mind me asking. Has karla ever like a agent?


Let us add a few more triples to the brain and check the result. This time Karla likes singing:

In [9]:
test_triple = {'subject': {'label': 'Karla', 'type': ['noun.person']},
               'predicate': {'label': 'like', 'type': ['verb.emotion']},
               'object': {'label': 'singing', 'type': ['noun.act']}}
capsule, throughts_json = talk.post_a_triple_label_and_type(test_triple, my_brain)

2021-11-15 14:46:26,483 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in statement: karla_like_singing [person_->_act])
2021-11-15 14:46:26,495 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Statement Novelty: 1 times, e.g. me on November,2021
2021-11-15 14:46:28,004 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Negation Conflicts: me on November,2021 about UNDERSPECIFIED
2021-11-15 14:46:28,025 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Gaps: 26 gaps as subject: e.g. own object - 15 gaps as object: e.g. be-friends-with person


In [10]:
test_triple = {'subject': {'label': 'Karla', 'type': ['noun.person']},
               'predicate': {'label': 'like', 'type': ['verb.emotion']},
               'object': {'label': 'pizza', 'type': ['noun.act']}}
capsule, throughts_json = talk.post_a_triple_label_and_type(test_triple, my_brain)

2021-11-15 14:46:29,369 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in statement: karla_like_pizza [person_->_act])
2021-11-15 14:46:29,394 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Entity Novelty: existing subject - new object 
2021-11-15 14:46:29,417 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Overlaps: 1 subject overlaps: e.g. me on November,2021 about singing - 0 object overlaps: e.g. ''
2021-11-15 14:46:30,963 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Negation Conflicts: me on November,2021 about UNDERSPECIFIED
2021-11-15 14:46:30,989 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Gaps: 26 gaps as subject: e.g. like-by agent - 15 gaps as object: e.g. know agent


In [11]:
for i in range(10):
    reply = replier.reply_to_statement(throughts_json, proactive=True, persist=True)
    print(reply)

I am glad to have learned something new. I did not know anybody who like pizza
I am glad to have learned something new. I had never heard about karla before!
I am curious. Has karla be friends with person?
Interesting! I did not know anybody who like pizza
I would like to know. What types of act orInstance like pizza do person orInstance usually like
This is news to me. I did not know anybody who like pizza
If you don't mind me asking. What types can karla like
I would like to know. Has karla be friends with person?
Interesting! I had never heard about karla before!
Cool! Did you know that karla also like singing


Launching the same SPARQL query to the brain will now give the following result:

![graph-db query](images/graph-db-query-2.png "GraphDB query for the triple result")

### Querying the brain

Since we have added properties to the brain, we can also ask questions. For this we need to make a triple with an empty slot: the variable. In the next example, we will ask what things Karla likes. Note that the API uses the labels of things and the label for Karla is automatically created by the posting function we used with *createLabel=True*. This adds the label "karla" in lowercase. We therfore also have to query with the label in lowercase.

In [12]:
test_triple = {'predicate': {'label': 'like', 'type': ['verb.emotion']}, 
               'subject': {'label': 'karla', 'type': ['agent']}, 
               'object': {'label': '', 'type': []}}
answer = talk.post_a_query(test_triple, my_brain)
pprint.pprint(answer)

{'chat': '1', 'turn': '1', 'author': 'me', 'utterance': '', 'utterance_type': <UtteranceType.QUESTION: 1>, 'position': '', 'context_id': '1', 'date': datetime.date(2021, 11, 15), 'place': '', 'place_id': '', 'country': '', 'region': '', 'city': '', 'objects': [], 'people': [], 'subject': {'label': 'karla', 'type': 'agent'}, 'predicate': {'label': 'like', 'type': 'verb.emotion'}, 'object': {'label': '', 'type': []}}
2021-11-15 14:46:36,581 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in question: karla_like_? [agent_->_])
{'question': {'author': 'me',
              'chat': '1',
              'city': '',
              'context_id': '1',
              'country': '',
              'date': '2021-11-15',
              'object': {'label': '', 'type': []},
              'objects': [],
              'people': [],
              'place': '',
              'place_id': '',
              'position': '',
              'predicate': {'label': 'like', 'type': 'verb.emotion'},
         

We can see the response value in the resulting JSON. In addition to the object labels (*olabel) returned, we also get additional properties on the sources of this information and their perspective on the claim that contained the triple. To humaise the repsonse, we can again call the *replier* but in this case we use the *reply_to_question* function:

In [13]:
reply = replier.reply_to_question(answer)
print(reply)

you told me karla maybe not verb.emotion pizza and that karla maybe not verb.emotion singing


You may wonder why the verbalisation inserted the modal phrase *maybe not*. This is because we have not specified the perspecive in the capsule. As you can see in the response, the value for Polarity is UNDERSPECIFIED which raises doubts about the belief. We will see below how the triple extractor handles this by processing real text rather than triples.

In [14]:
test_triple = {'predicate': {'label': 'like', 'type': ['verb.emotion']}, 
               'subject': {'label': 'fred', 'type': ['agent']}, 
               'object': {'label': '', 'type': []}}
answer = talk.post_a_query_and_verbalise_answer(test_triple, replier, my_brain)
print(answer)

{'chat': '1', 'turn': '1', 'author': 'me', 'utterance': '', 'utterance_type': <UtteranceType.QUESTION: 1>, 'position': '', 'context_id': '1', 'date': datetime.date(2021, 11, 15), 'place': '', 'place_id': '', 'country': '', 'region': '', 'city': '', 'objects': [], 'people': [], 'subject': {'label': 'fred', 'type': 'agent'}, 'predicate': {'label': 'like', 'type': 'verb.emotion'}, 'object': {'label': '', 'type': []}}
2021-11-15 14:46:39,952 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in question: fred_like_? [agent_->_])
I don't know


Again, we see that the perspective is not set and therefore the pespective of the source results in strange phrasings. In the notebook: *lets-chat-with-brain-lenk.ipynb* we will use more advanced Natural Language Processing to extract such perspectives from the utterances within a conversational interaction setting.

## End of notebook